In [1]:
dia = ''

In [2]:
# Parameters
dia = "2024-04-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
from scripts.AtualizaUltimosJogos import *
import warnings
warnings.filterwarnings('ignore')

# Pegando os jogos do dia para atualizar
jogos_passados = pd.read_csv(f'jogos_do_dia/{dia}.csv')
dia_jogos = jogos_passados['Date'].iloc[0]
id_jogos = jogos_passados['Fixture ID'].unique().tolist()
print(f'Jogos do dia {dia_jogos}\n')

# Buscando os resultados
results = atualiza_ultimos_jogos(id_jogos = id_jogos)

# Merjando os resultados no dataset de jogos do dia
df_resultados = pd.merge(left=jogos_passados, right=results[['Fixture ID', 'Home_Pts', 'Away_Pts', 'Status']], on='Fixture ID', how='left')
df_resultados['Status'].replace(np.nan, 'Não iniciado', inplace=True)

# Organizando as colunas do df_resultados
df_resultados = df_resultados[['Fixture ID', 'Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', 'Status']]
df_resultados = df_resultados.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
df_resultados.sort_values('Time', inplace=True)
df_resultados.reset_index(inplace=True, drop=True)
# Criando df com jogos que ainda não terminaram
df_restantes = df_resultados[(df_resultados['Status'] != 'Após Tempo Extra') & (df_resultados['Status'] != 'Encerrado') & (df_resultados['Status'] != 'Após Pênaltis')]
df_restantes = df_restantes[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Status']]

# Limpando o df_resultados
df_resultados = df_resultados[(df_resultados['Status'] == 'Após Tempo Extra') | (df_resultados['Status'] == 'Encerrado') | (df_resultados['Status'] == 'Após Pênaltis')]
df_resultados = df_resultados[['Date', 'League', 'Time', 'Home', 'Away', 'Home_Pts', 'Away_Pts', 'Odds_H',
       'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A']]

# Exclui os registros que já existem no dataset da temporada
temporada = pd.read_csv('data/Entire_Season.csv')
n_antes = len(temporada)
temporada = pd.concat([temporada, df_resultados], ignore_index=True)
temporada = temporada.drop_duplicates(subset=['Date', 'Home', 'Away'], keep='first')
temporada.reset_index(drop=True, inplace=True)
temporada.to_csv('data/Entire_Season.csv', index=False)

print(f'Dataset de temporada atualizado com {len(temporada) - n_antes} jogos.\n')
print('----------------------------------------------------------\n')
print(f'Atenção! {len(df_restantes)} jogos não finalizados.\n')
display(df_restantes)

Jogos do dia 2024-04-28



  0%|                                                                                          | 0/115 [00:00<?, ?it/s]

  1%|▋                                                                                 | 1/115 [00:01<01:54,  1.01s/it]

  2%|█▍                                                                                | 2/115 [00:01<01:48,  1.04it/s]

  3%|██▏                                                                               | 3/115 [00:02<01:29,  1.25it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 114/115 [01:27<00:00,  1.22it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 115/115 [01:28<00:00,  1.19it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 115/115 [01:28<00:00,  1.30it/s]

Dataset de temporada atualizado com 115 jogos.

----------------------------------------------------------

Atenção! 0 jogos não finalizados.



,Date,League,Time,Home,Away,Home_Pts,Away_Pts,Status
